In [4]:
import torch
import torch.nn as nn
import pickle
import pandas as pd

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [42]:
with open('train_set_kaggle.pkl', 'rb') as f:
    X_train, Y_train = pickle.load(f)

with open('test_set_kaggle.pkl', 'rb') as f:
    X_test = pickle.load(f)
X_train = torch.from_numpy(X_train).to(dtype=torch.float32)
X_test = torch.from_numpy(X_test).to(dtype=torch.float32)
Y_train = torch.from_numpy(Y_train).to(dtype=torch.float32)

In [56]:
import numpy as np


class ANN(nn.Module):
    def __init__(self, weight_init_scheme=None, num_hidden_layers=3, num_hidden_size=5, act_func="RELU",
                 input_feature=32):
        super(ANN, self).__init__()
        self.sequential = nn.Sequential().to(dtype=torch.float32)
        if act_func == "RELU":
            self.act = nn.ReLU()
        else:
            self.act = nn.Tanh()
        for i in range(num_hidden_layers - 1):
            self.sequential.add_module('dropout{}'.format(i), nn.Dropout(p=0.5))
            linear = nn.Linear(input_feature, num_hidden_size).to(dtype=torch.float32)
            if weight_init_scheme == "Xavier":
                nn.init.xavier_uniform_(linear.weight)
            else:
                nn.init.kaiming_uniform_(linear.weight, nonlinearity='relu')
            self.sequential.add_module('linear{}'.format(i), linear)
            if act_func == "RELU":
                self.sequential.add_module('act{}'.format(i), self.act)
            input_feature = num_hidden_size

        self.sequential.add_module('layer_norm{}'.format(num_hidden_layers - 1),
                                   nn.Linear(num_hidden_size, 1))

    def forward(self, input):
        output = self.sequential(input)
        return output


def train(network,criterion, optimizer, X, y):
    # Train our network on a given batch of X and y.
    # We first need to run forward to get all layer activations.
    # Then we can run layer.backward going from last to first layer.
    # After we have called backward for all layers, all Dense layers have already made one gradient step.

    # Get the layer activations
    network.train()
    optimizer.zero_grad()
    output = network(X)
    # Compute the loss and the initial gradient
    y = torch.unsqueeze(y,1)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    return loss

from tqdm import trange

def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


In [51]:
def binary_err(y_pred, y_test):
    wrong_results_sum = (y_pred != y_test).sum().float()
    err = wrong_results_sum/y_test.shape[0]
    return err


def predict_proba(X, model, num_samples):
    preds = []
    for i in range(num_samples):
      pred = nn.Sigmoid()(model(X))
      preds.append(pred)
    return torch.stack(preds).mean()

def predict_class(X, model, num_samples):
    proba_preds = predict_proba(X, model, num_samples)
    return proba_preds.reshape(-1).round()

In [57]:
from IPython.display import clear_output

#,10,25,50,100
learning_rate= 1e-3
input_shape = X_train.shape[-1]

network = ANN(weight_init_scheme="He",num_hidden_layers=3, num_hidden_size=300,act_func="RELU",input_feature=input_shape)
train_log = []
test_log = []
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=1e-5)
for epoch in range(25):
    for x_batch, y_batch in iterate_minibatches(X_train, Y_train, batchsize=128, shuffle=True):
          train(network, criterion, optimizer, x_batch, y_batch)

    network.eval()
    # y_pred = nn.Sigmoid()(network(X_train)).reshape(-1).round()
    # wrong_results_sum = (y_pred != Y_train).sum().float()
    # err = wrong_results_sum/Y_train.shape[0]
    y_pred = predict_class(X_train, network, 50)
    wrong_results_sum = (y_pred != Y_train).sum().float()
    err = wrong_results_sum/Y_train.shape[0]
    train_log.append(err)
# acc_test = np.mean((network(X_test).reshape(-1).detach().numpy().round() != Y_test.numpy()))
print("Train error:", train_log[-1])
# print("Val error:", acc_test)

Train error: tensor(0.2406)


In [ ]:
y_pred = np.squeeze(network(X_test).detach().numpy().round())
id = np.arange(len(y_pred))+1
data_frame = pd.DataFrame({"Prediction": y_pred}, index=id)
data_frame.insert(0, "ID", id)
data_frame.to_csv("Kaggle_pred_DNN_5000_MCdropout", index=False)
print(data_frame)

          ID  Prediction
1          1         0.0
2          2         0.0
3          3         0.0
4          4         0.0
5          5         0.0
...      ...         ...
23838  23838         0.0
23839  23839         0.0
23840  23840         1.0
23841  23841         0.0
23842  23842         0.0

[23842 rows x 2 columns]


In [21]:
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Flatten, SpatialDropout2D, SpatialDropout1D, AlphaDropout
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import StandardScaler    

In [2]:
batch_size = 128
num_classes = 10
epochs = 12

In [40]:
with open('train_set_kaggle.pkl', 'rb') as f:
    X_train, Y_train = pickle.load(f)

with open('test_set_kaggle.pkl', 'rb') as f:
    X_test = pickle.load(f)

In [41]:
initializer = keras.initializers.HeNormal()
model = keras.models.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer=initializer))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer=initializer))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(1, activation="softmax", kernel_initializer=initializer))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Nadam(lr=1e-5),
              metrics=['accuracy'])
model.fit(X_train, Y_train,batch_size=batch_size, epochs=1000)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


Epoch 1/1000
196/196 [==============================] - 2s 5ms/step - loss: 0.9011 - accuracy: 0.2406
Epoch 2/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.6474 - accuracy: 0.2406
Epoch 3/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.6011 - accuracy: 0.2406
Epoch 4/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.5622 - accuracy: 0.2406
Epoch 5/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.5361 - accuracy: 0.2406
Epoch 6/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.5149 - accuracy: 0.2406
Epoch 7/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.4936 - accuracy: 0.2406
Epoch 8/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.4876 - accuracy: 0.2406
Epoch 9/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.4731 - accuracy: 0.2406
Epoch 10/1000
196/196 [==============================] - 1s 5ms/step - loss: 0.471

In [14]:
def predict_proba(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.stack(preds).mean(axis=0)
     
def predict_class(X, model, num_samples):
    proba_preds = predict_proba(X, model, num_samples)
    return np.argmax(proba_preds, axis=1)

In [19]:
y_pred = predict_class(X_train, model, 100)
acc = np.mean(y_pred == Y_train)
print(acc)

0.75936


In [16]:
y_pred = predict_class(X_test, model, 100)
id = np.arange(len(y_pred))+1
data_frame = pd.DataFrame({"Prediction": y_pred}, index=id)
data_frame.insert(0, "ID", id)
data_frame.to_csv("Kaggle_pred_DNN_5000_MCdropout", index=False)
print(data_frame)

          ID  Prediction
1          1           0
2          2           0
3          3           0
4          4           0
5          5           0
...      ...         ...
23838  23838           0
23839  23839           0
23840  23840           0
23841  23841           0
23842  23842           0

[23842 rows x 2 columns]
